In [1]:
import os
import torch

import utils

%load_ext autoreload
%autoreload 2

CUDA_LAUNCH_BLOCKING=1
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
print(os.listdir("../data/"))


['kawiki.txt', 'data3.txt', 'data.txt', 'data_115000.txt', 'data2.txt']


In [2]:
from utils import GeorgianLanguageDatasetLoader
import gc
gc.collect()
torch.cuda.empty_cache()

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# using cpu because gpu is not enough
device = torch.device('cpu')
dataset = GeorgianLanguageDatasetLoader("../data/data_115000.txt", 5, device)

In [3]:
vocab = dataset.get_vocabulary()
print(vocab.get_itos()[:20])

['<unk>', ',', '.', 'და', ')', '(', '„', '“', 'წელს', '—', 'იყო', 'წლის', ':', 'რომელიც', 'შემდეგ', 'რომ', 'მისი', '``', 'ამ', "''"]


In [4]:
train_data, valid_data, test_data = dataset.get_data()
train_data[:10]

tensor([ 910,  644,  517,  691, 9915,    1,   13,  154,  509, 1907])

In [5]:
' '.join([vocab.get_itos()[i] for i in train_data[40:60]])

'მინდა გემორჩილო და ბატონად გაღიაროვო “ . მსოფლიო ბანკი დანიას ევროკავშირში ბიზნესის ყველაზე ადვილად კეთების ადგილად მიიჩნევს . ქაოსი ('

In [6]:
text_pipeline = dataset.get_text_pipeline()
text_pipeline("შავი კაცი მიდიოდა")

[364, 342, 4857]

In [7]:
train_data_batched, val_data_batched = dataset.get_batched_data()

In [8]:
train_data_batched.shape, train_data_batched.device

(torch.Size([5, 158221]), device(type='cpu'))

In [9]:
x, y = utils.get_batch(train_data_batched, 1)

In [10]:
x.shape, y.shape, train_data_batched.shape

(torch.Size([5, 10]), torch.Size([5, 10]), torch.Size([5, 158221]))

In [11]:
x.storage().data_ptr() == train_data_batched.storage().data_ptr()

True

In [12]:
x[0], y[0]

(tensor([ 644,  517,  691, 9915,    1,   13,  154,  509, 1907, 3464]),
 tensor([ 517,  691, 9915,    1,   13,  154,  509, 1907, 3464, 1095]))

In [24]:
%reload_ext autoreload

model = utils.LSTMModel(50, 300, len(vocab), device, 1).cuda()
model = model.to(device)
utils.train_loop(model, train_data_batched, batch_size=5)

/home/misho/anaconda3/envs/nlp_final_project/lib/python3.7/site-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


{'epoch': 0, 'batch': 0, 'loss': 11.636711120605469}
{'epoch': 1, 'batch': 0, 'loss': 11.582306861877441}
{'epoch': 2, 'batch': 0, 'loss': 11.527762413024902}
{'epoch': 3, 'batch': 0, 'loss': 11.47675895690918}
{'epoch': 4, 'batch': 0, 'loss': 11.431130409240723}
{'epoch': 5, 'batch': 0, 'loss': 11.371769905090332}
{'epoch': 6, 'batch': 0, 'loss': 11.29650592803955}
{'epoch': 7, 'batch': 0, 'loss': 11.198358535766602}
{'epoch': 8, 'batch': 0, 'loss': 11.116037368774414}
{'epoch': 9, 'batch': 0, 'loss': 11.004416465759277}
{'epoch': 10, 'batch': 0, 'loss': 10.78904914855957}
{'epoch': 11, 'batch': 0, 'loss': 10.543282508850098}
{'epoch': 12, 'batch': 0, 'loss': 10.217506408691406}
{'epoch': 13, 'batch': 0, 'loss': 9.797294616699219}
{'epoch': 14, 'batch': 0, 'loss': 9.305048942565918}
{'epoch': 15, 'batch': 0, 'loss': 8.8091459274292}
{'epoch': 16, 'batch': 0, 'loss': 8.248229026794434}
{'epoch': 17, 'batch': 0, 'loss': 7.704070568084717}
{'epoch': 18, 'batch': 0, 'loss': 7.143138408660

In [25]:
text = utils.generate_text(model,device, dataset.vocab, 'საგარეო პოლიტიკა', 20)

In [26]:
print(text)

საგარეო პოლიტიკა ჰიროსიმისა აღმავლობა ვუკოვარზე დედოფალ ლიკვიდატორებისა , კირქვისა და და ანდეზიტ-ბაზალტის საბადო . მამია IV გურიელი სამრეკლოს სამრეკლოს და გურიელი სამრეკლოს


In [16]:
torch.cuda.empty_cache()

In [17]:
# geo_model = utils.GeorgianFastTextModel(load=True)
# geo_model.train("../data/data_115000.txt")

In [18]:
# embeddings = torch.FloatTensor(geo_model.get_model().wv.vectors)
# embeddings.shape

In [19]:
# %reload_ext autoreload
#
# model = utils.LSTMModel(128, 600, len(vocab), device, num_layers=1, embeddings=embeddings).cuda()
# model = model.to(device)
# utils.train_loop(model, train_data_batched, batch_size=5)

In [20]:
# import torchtext
# v = torchtext.vocab.vocab(geo_model.get_model().wv.key_to_index, specials=['<unk>'])
# v.set_default_index(v['<unk>'])
# # dataset.vocab.load_state_dict(geo_model.get_model().wv.key_to_index)
# text = utils.generate_text(model,device, v, 'საგაერო პოლიტიკა', 10)

In [21]:
# print(text)